In [5]:
from sklearn.preprocessing import RobustScaler
import pandas as pd
import calendar
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
from joblib import delayed, Parallel
from sklearn.metrics import silhouette_score
from tqdm import tqdm
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.manifold import TSNE


In [42]:
average_returns_per_month = pd.read_csv("/Users/sebasleen/Desktop/Data Science/Data/average_returns_per_month.csv")
cluster_monthly_returns = pd.read_csv("/Users/sebasleen/Desktop/Data Science/Data/cluster_monthly_returns.csv")
kmeans_interpolated =pd.read_csv("/Users/sebasleen/Desktop/Data Science/Data/kmeans_interpolated.csv")
momentum_strategy_returns =pd.read_csv("/Users/sebasleen/Desktop/Data Science/Data/momentum_strategy_returns.csv")
monthly_cluster_factors_values =pd.read_csv("/Users/sebasleen/Desktop/Data Science/Data/monthly_cluster_factor_values.csv")
scaled_demeaned_factors = pd.read_csv("/Users/sebasleen/Desktop/Data Science/Data/scaled_demeaned_factors.csv")

print(average_returns_per_month)
print(cluster_monthly_returns)
print(kmeans_interpolated)
print(momentum_strategy_returns)
print(monthly_cluster_factors_values)
print(scaled_demeaned_factors)


# Extract the dates from the keys of the dictionary
#dates = list(cluster_momentum_strategy_returns.keys())
#
## Extract cluster names
#clusters = list(cluster_momentum_strategy_returns[dates[0]].keys())
#
## Create an empty DataFrame
#momstrategies = pd.DataFrame(columns=clusters, index=dates)
#
## Populate the DataFrame with values from the dictionary
#for date, returns in cluster_momentum_strategy_returns.items():
#    momstrategies.loc[date] = [returns[cluster] for cluster in clusters]
#
#print(momstrategies)
#momstrategies.fillna(0, inplace=True)
#print(momstrategies)

       Unnamed: 0 Unnamed: 1         0
0      1963-07-31     r_size  0.118791
1      1963-07-31    r_value -0.072649
2      1963-07-31     r_prof -0.004062
3      1963-07-31      r_dur  0.070475
4      1963-07-31  r_valprof -0.059912
...           ...        ...       ...
31861  2019-12-31     r_ciss  0.140819
31862  2019-12-31    r_price -0.247168
31863  2019-12-31      r_age  0.183511
31864  2019-12-31    r_shvol  0.102215
31865  2019-12-31      r_ipo -0.435967

[31866 rows x 3 columns]
     Unnamed: 0         1         2         3         4
0    1963-07-31  0.033254  0.000000  0.201455 -0.153184
1    1963-08-31  0.048903  0.049739 -0.061865 -0.012725
2    1963-09-30  0.264533  0.256336  0.000874 -0.001177
3    1963-10-31  0.058375 -0.000474 -0.090637 -0.145163
4    1963-11-30  0.092528 -0.962351  0.000000  0.647453
..          ...       ...       ...       ...       ...
673  2019-08-31  0.477560 -0.100603 -0.010560 -0.294354
674  2019-09-30 -0.148862  0.000000 -0.214281  1.073770
67

In [47]:
def compute_momentum_strategy_returns(scaled_demeaned_factors, lookback_period=12):
    momentum_strategy_returns = {}

    # Iterate over each month
    for month, factors_in_month in scaled_demeaned_factors.items():
        # Calculate the average return for each cluster over the past lookback period
        cluster_returns = {}
        for cluster_id, cluster_factors in factors_in_month.items():
            cluster_returns[cluster_id] = np.mean([factor for factor in cluster_factors.values()])

        # Determine long and short positions
        long_clusters = [cluster_id for cluster_id, cluster_return in cluster_returns.items() if cluster_return > 0]
        short_clusters = [cluster_id for cluster_id, cluster_return in cluster_returns.items() if cluster_return < 0]

        # Compute the return on the momentum strategy
        momentum_return = 0
        for cluster_id in long_clusters:
            momentum_return += np.mean([factor for factor in factors_in_month[cluster_id].values()])
        for cluster_id in short_clusters:
            momentum_return -= np.mean([factor for factor in factors_in_month[cluster_id].values()])

        momentum_strategy_returns[month] = momentum_return

    return momentum_strategy_returns

# Call the function to compute momentum strategy returns
momentum_strategy_returns = compute_momentum_strategy_returns(scaled_demeaned_factors)

print(momentum_strategy_returns)


AttributeError: 'str' object has no attribute 'values'